In [27]:
import random
import math
import time
import arcade
import psycopg2

In [28]:
dbname = 'rl_dofus'
user = 'nono'
password = 'azeAZE123'
host = 'localhost'

connection = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

cursor = connection.cursor()

In [29]:
#cursor.execute("DROP TABLE IF EXISTS qtable")
cursor.execute("CREATE TABLE IF NOT EXISTS qtable (states CHAR(562), actions INT, reward FLOAT, PRIMARY KEY (states, actions));")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_state ON qtable (states);")
connection.commit()

In [30]:
def get_pos_at_range(r):
    
    list_pos = []
    
    for i in range(r + 1):
        if i != 0:
            list_pos.append((0, i))
            list_pos.append((0, -i))
    
        for j in range(r - i):
            list_pos.append((1 + j, i))
            list_pos.append((-1 - j, i))
            
            if i != 0:
                list_pos.append((1 + j, -i))
                list_pos.append((-1 - j, -i))
    
    return list_pos

In [31]:
"""
#  #  #  #  #  #  1  #  #  #  #  #  #
#  #  #  #  #  2  3  4  #  #  #  #  #
#  #  #  #  5  6  7  8  9  #  #  #  #
#  #  #  10 11 12 13 14 15 16 #  #  #
#  #  17 18 19 20 21 22 23 24 25 #  #
#  26 27 28 29 30 31 32 33 34 35 36 #
37 38 39 40 41 42 -- 44 45 46 47 48 49
#  50 51 52 53 54 55 56 57 58 59 60 #
#  #  61 62 63 64 65 66 67 68 69 #  #
#  #  #  70 71 72 73 74 75 76 #  #  #
#  #  #  #  77 78 79 80 81 #  #  #  #
#  #  #  #  #  82 83 84 #  #  #  #  #
#  #  #  #  #  #  85 #  #  #  #  #  #
"""

END_TURN = -1
ACTIONS = get_pos_at_range(6)

ACTIONS.append(END_TURN)
print(len(ACTIONS))

85


In [32]:
def insert_new_state(state):
    for action in range(len(ACTIONS)):
        cursor.execute("INSERT INTO qtable(states, actions, reward) VALUES (%s, %s, 0)", (state, action))
    connection.commit()

In [33]:
def get_max_action_state(state):
    cursor.execute("SELECT actions FROM qtable WHERE states = %s ORDER BY reward DESC", (state,))
    returned = cursor.fetchall()
    if len(returned) == 0:
        return None
    return returned[0][0]    

In [34]:
def add_reward(state, action, reward):
    cursor.execute("UPDATE qtable SET reward = reward + %s WHERE states = %s AND actions = %s RETURNING reward", (reward, state, action))
    
    returned = cursor.fetchall()
    connection.commit()
    
    return returned[0][0]

In [35]:
def compute_reward(state, action, reward, last):
    cursor.execute("UPDATE qtable SET reward = reward + %s *(%s + %s * %s - reward) WHERE states = %s AND actions = %s RETURNING reward", (learning_rate, reward, decrease_factor, last, state, action))
    
    returned = cursor.fetchall()
    connection.commit()
    
    return returned[0][0]

In [36]:
REWARD_MOB = 16
REWARD_END_TURN = -1
REWARD_END_TURN_WITHOUT_ACTION = -16
REWARD_MOVE = -2
REWARD_IMPOSSIBLE = -128
REWARD_END_FIGHT = 64

EMPTY_CELL = 0
WALL = 1
USABLE_CELL = 2
PLAYER = 3
MONSTER = 4
INVO = 5
BOSS = 6
UNDEFINED = -1

MAX_PA = 6
MAX_PM = 3

learning_rate = 1
decrease_factor = 0.5
random_action = 0.01

In [37]:
l = 20
h = 14

base_x = 235
base_y = 5

step_x = 130
step_y = 65

box_size = 30

dict_swap_dofus_to_pos = {}
dict_swap_pos_to_dofus  = {}

dict_swap_dofus_to_array_index  = {}
dict_swap_pos_to_array_index  = {}

dict_swap_array_index_to_dofus  = []
dict_swap_array_index_to_pos  = []
index = 0

for j in range(h):
    
    x_start = 13 - j
    
    for i in range(l):
        
        dict_swap_dofus_to_pos[(j, i, False)] = (x_start + i, i + j)
        dict_swap_dofus_to_pos[(j, i, True)] = (x_start + i, i + j + 1)
        
        dict_swap_pos_to_dofus[(x_start + i, i + j)] = (j, i, False)
        dict_swap_pos_to_dofus[(x_start + i, i + j + 1)] = (j, i, True)
        
        dict_swap_dofus_to_array_index[(j, i, False)] = index
        dict_swap_pos_to_array_index[(x_start + i, i + j)] = index
        
        dict_swap_array_index_to_dofus.append((j, i, False))
        dict_swap_array_index_to_pos.append((x_start + i, i + j))
        index += 1

        dict_swap_dofus_to_array_index[(j, i, True)] = index
        dict_swap_pos_to_array_index[(x_start + i, i + j + 1)] = index
        dict_swap_array_index_to_dofus.append((j, i, True))
        dict_swap_array_index_to_pos.append((x_start + i, i + j + 1))
        
        index += 1
        

In [38]:
def get_pos_at_range_possible(map_array, start, r):
    
    list_pos = get_pos_at_range(r)
    full_list_pos = []
    for i in range(len(list_pos)):
        
        list_pos[i] = list_pos[i][0] + start[0], list_pos[i][1] + start[1]
        
        if list_pos[i] not in dict_swap_pos_to_array_index.keys():
            continue
    
        if map_array[dict_swap_pos_to_array_index[list_pos[i]]] == USABLE_CELL:
            full_list_pos.append(list_pos[i])

    return full_list_pos

In [39]:
def get_map(path):
    
    dict_map = {}
    array_map = []
    list_state_map = []
    
    with open(path, 'r') as f:
        lines = f.readlines()
        
        for line in lines:
            array_map.append(list(map(int, line.split(','))))
    
    for pos in dict_swap_pos_to_dofus.keys():
        dict_map[pos] = array_map[pos[0]][pos[1]]
        
    for i in range(h):
        for j in range(l):
            list_state_map.append(dict_map[dict_swap_dofus_to_pos[(i, j, False)]])
            list_state_map.append(dict_map[dict_swap_dofus_to_pos[(i, j, True)]])
    
    return list_state_map

In [40]:
def get_random_start_map(map_array):
    
    list_keys_player = []
    list_keys_mob = []
    
    for key in range(len(map_array)):
        if map_array[key] == 3:
            list_keys_player.append(key)
        elif map_array[key] == 4:
            list_keys_mob.append(key)

    i = random.randint(0, len(list_keys_player) - 1)
    list_keys_player.remove(list_keys_player[i])

    for _ in range(4):
        i = random.randint(0, len(list_keys_mob) - 1)
        list_keys_mob.remove(list_keys_mob[i])

    for pos in list_keys_player:
        map_array[pos] = 2

    for pos in list_keys_mob:
        map_array[pos] = 2

    return map_array

In [41]:
def get_player_pos(map_array):
    
    for pos in range(len(map_array)):
        if map_array[pos] == 3:
            return dict_swap_array_index_to_pos[pos]
        
    return -1

In [42]:
def mob_turn(map_array, invo=False):
    
    list_mob = []
    
    for pos in range(len(map_array)):
        if map_array[pos] == MONSTER:
            list_mob.append(pos)
    
    for mob in list_mob:
        
        possible_pos = get_pos_at_range_possible(map_array, dict_swap_array_index_to_pos[mob], 3)
        
        if len(possible_pos) == 0:
            continue

        map_array[mob] = USABLE_CELL
        new_pos = dict_swap_pos_to_array_index[possible_pos[random.randint(0, len(possible_pos) - 1)]]
        map_array[new_pos] = MONSTER

    return map_array

In [43]:
def map_contains_mob(map_array):

    for pos in range(len(map_array)):
        if map_array[pos] == MONSTER or map_array[pos] == INVO or map_array[pos] == BOSS:
            return True

    return False

In [44]:
def ldv(map_array, from_pos, to_pos):
    
    diff = [to_pos[0] - from_pos[0], to_pos[1] - from_pos[1]]
    tmp_from = [from_pos[0], from_pos[1]]
    
    if abs(diff[0]) > abs(diff[1]):
        index_factor = 0
        index_float = 1
    else:
        index_factor = 1
        index_float = 0

    factor = diff[0] / abs(diff[index_factor]), diff[1] / abs(diff[index_factor])
    
    while tmp_from[0] != to_pos[0] or tmp_from[1] != to_pos[1]:
        

        if not float(tmp_from[index_float]).is_integer():
            
            up = [0, 0]
            down = [0, 0]
            
            up[index_factor] = tmp_from[index_factor]
            down[index_factor] = tmp_from[index_factor]
            
            up[index_float] = math.ceil(tmp_from[index_float])
            down[index_float] = math.floor(tmp_from[index_float])
            
            if tuple(up) in dict_swap_pos_to_array_index.keys():
                up_index = dict_swap_pos_to_array_index[tuple(up)]            
                if map_array[up_index] == WALL or map_array[up_index] == MONSTER or map_array[up_index] == BOSS or map_array[up_index] == INVO:
                    return False
            
            if tuple(down) in dict_swap_pos_to_array_index.keys():
                down_index = dict_swap_pos_to_array_index[tuple(down)]
                if map_array[down_index] == WALL or map_array[down_index] == MONSTER or map_array[down_index] == BOSS or map_array[down_index] == INVO:
                    return False
        else:
            
            tmp_index = dict_swap_pos_to_array_index[tuple(tmp_from)]
            
            if map_array[tmp_index] == WALL or map_array[tmp_index] == MONSTER or map_array[tmp_index] == BOSS or map_array[tmp_index] == INVO:
                return False
            
        tmp_from[0] += factor[0]
        tmp_from[1] += factor[1]
    
    return True

In [45]:
def can_move(map_array, from_pos, to_pos):
    
    diff = [to_pos[0] - from_pos[0], to_pos[1] - from_pos[1]]
    tmp_from = [from_pos[0], from_pos[1]]
    
    if abs(diff[0]) > abs(diff[1]):
        index_factor = 0
        index_float = 1
    else:
        index_factor = 1
        index_float = 0

    factor = diff[0] / abs(diff[index_factor]), diff[1] / abs(diff[index_factor])
    
    while tmp_from[0] != to_pos[0] and tmp_from[1] != to_pos[1]:
        if not float(tmp_from[index_float]).is_integer():
            
            up = [0, 0]
            down = [0, 0]
            
            up[index_factor] = tmp_from[index_factor]
            down[index_factor] = tmp_from[index_factor]
            
            up[index_float] = math.ceil(tmp_from[index_float])
            down[index_float] = math.floor(tmp_from[index_float])
            
            if tuple(up) in dict_swap_pos_to_array_index.keys():
                up_index = dict_swap_pos_to_array_index[tuple(up)]            
                if map_array[up_index] == WALL or map_array[up_index] == MONSTER or map_array[up_index] == BOSS or map_array[up_index] == INVO:
                    return False
            
            if tuple(down) in dict_swap_pos_to_array_index.keys():
                down_index = dict_swap_pos_to_array_index[tuple(down)]
                if map_array[down_index] == WALL or map_array[down_index] == MONSTER or map_array[down_index] == BOSS or map_array[down_index] == INVO:
                    return False
            
        else:
            
            tmp_index = dict_swap_pos_to_array_index[tuple(tmp_from)]
            
            if map_array[tmp_index] != USABLE_CELL:
                return False

        tmp_from[0] += factor[0]
        tmp_from[1] += factor[1]
    
    return True

In [46]:
def count_pm_to_pos(map_array, player_pos, expected_pos, max_pm):
    
    diff = expected_pos[0] - player_pos[0], expected_pos[1] - player_pos[1]
    dist = abs(diff[0]) + abs(diff[1])
    
    if dist < max_pm and can_move(map_array, player_pos, expected_pos):
        return dist

    return -1

In [47]:
class MapState:
    
    def __init__(self, pa, pm, map, map_index):

        self.map = map
        self.map_index = map_index
        self.pa = pa
        self.pm = pm
        
    def clone(self):
        
        new_map = []
        
        for i in range(len(self.map)):
            new_map.append(self.map[i])
        
        return MapState(self.pa, self.pm, new_map, self.map_index)

    @staticmethod
    def generate():

        #self.map_index = random.randint(0, 4)
        map_index = 0
        map = get_random_start_map(get_map(f"./data/map_array/map_{map_index + 1}.txt"))
        
        return MapState(MAX_PA, MAX_PM, map, map_index)

    @property
    def state(self):

        state = ""
        for i in range(len(self.map)):
            state += str(self.map[i])

        state += str(self.pa)
        state += str(self.pm)

        return state

In [48]:
class Environment:
    
    def do(self, map_state, action):
        
        clicked = ACTIONS[action]
        player_pos = get_player_pos(map_state.map)
        new_state = map_state.clone()
        
        if player_pos == -1:
            exit()
        
        if clicked == END_TURN:
            reward = REWARD_END_TURN
            if new_state.pm == MAX_PM and new_state.pa == MAX_PA:
                reward = REWARD_END_TURN_WITHOUT_ACTION
            new_state.pa = MAX_PA
            new_state.pm = MAX_PM
            new_state.map = mob_turn(new_state.map, invo=False)
            return reward, new_state
        
        clicked_pos = player_pos[0] + clicked[0], player_pos[1] + clicked[1]
        
        if clicked_pos not in dict_swap_pos_to_array_index.keys():
            return REWARD_IMPOSSIBLE, new_state
        
        clicked_index = dict_swap_pos_to_array_index[clicked_pos]

        if new_state.map[clicked_index] == EMPTY_CELL or new_state.map[clicked_index] == WALL or new_state.map[clicked_index] == UNDEFINED:
            return REWARD_IMPOSSIBLE, new_state
        
        elif new_state.map[clicked_index] == MONSTER or new_state.map[clicked_index] == INVO or new_state.map[clicked_index] == BOSS:
            
            if new_state.pa < 3:
                return REWARD_IMPOSSIBLE, new_state
            
            if not ldv(new_state.map, player_pos, clicked_pos):
                return REWARD_IMPOSSIBLE, new_state
            
            new_state.map[clicked_index] = USABLE_CELL
            new_state.pa -= 3
            
            if not map_contains_mob(new_state.map):
                return REWARD_END_FIGHT, new_state
            return REWARD_MOB, new_state
        
        elif new_state.map[clicked_index] == USABLE_CELL:
            used = count_pm_to_pos(new_state.map, player_pos, clicked_pos, new_state.pm)
            
            if used == -1:
                return REWARD_IMPOSSIBLE, new_state
            
            new_state.map[clicked_index] = PLAYER
            new_state.map[dict_swap_pos_to_array_index[player_pos]] = USABLE_CELL
            new_state.pm -= used
            return REWARD_MOVE, new_state


In [49]:
class Agent:
    
    def __init__(self):
        self.env = Environment()
        self.map_state = MapState.generate()
        self.score = 0
        self.list_actions = []

    def reset(self):
        self.map_state = MapState.generate()
        self.score = 0
        if len(self.list_actions) > 0:
            self.back_propagation()
        self.list_actions = []
        
    def do_all(self, state):
        for action in range(len(ACTIONS)):
            reward, new_state = self.env.do(self.map_state, action)
            add_reward(state, action, reward)
        
    def best_action(self, state):

        action = get_max_action_state(state)
        if action is None:
            insert_new_state(state)
            self.do_all(state)
            action = get_max_action_state(state)

        if random.uniform(0, 1) < random_action:
            return random.randint(0, len(ACTIONS) - 1), True

        return action, False
    
    def back_propagation(self):
        
        last = add_reward(self.list_actions[-1][0], self.list_actions[-1][1], self.list_actions[-1][2])
        for i in range(len(self.list_actions) - 2, -1, -1):
            last = compute_reward(self.list_actions[i][0], self.list_actions[i][1], self.list_actions[i][2], last)
    
    def do(self):
        action, randomed = self.best_action(self.map_state.state)
        
        reward, new_state = self.env.do(self.map_state, action)
        
        if randomed and self.map_state.state == new_state.state:
            return reward

        self.list_actions.append((self.map_state.state, action, reward))
        self.score += reward

        self.map_state = new_state

        return reward

In [50]:
class DofusStatusWindow(arcade.Window):
    
    def __init__(self):
        super().__init__(2256 // 2, 1504 // 2, "DofusStatus")
        arcade.set_background_color(arcade.color.BLACK)
        self.agent = Agent()
        
    def draw_box(self, x, y, color):
        arcade.draw_rectangle_filled(x, 1504 // 2 - y, box_size // 2, box_size // 2, color)
        
    def on_draw(self):
        arcade.start_render()
        for pos in range(len(self.agent.map_state.map)):
            if self.agent.map_state.map[pos] != 0:

                dofus_pos = dict_swap_array_index_to_dofus[pos]

                x = base_x + step_x * dofus_pos[0]
                y = base_y + step_y * dofus_pos[1]
                
                if dofus_pos[2]:
                    x += step_x // 2
                    y += step_y // 2
                
                x = x // 2
                y = y // 2
                
                if self.agent.map_state.map[pos] == 1:
                    self.draw_box(x, y, arcade.color.GRAY)
                elif self.agent.map_state.map[pos] == 2:
                    self.draw_box(x, y, arcade.color.GREEN)
                elif self.agent.map_state.map[pos] == 3:
                    self.draw_box(x, y, arcade.color.BLUE)
                elif self.agent.map_state.map[pos] == 4:
                    self.draw_box(x, y, arcade.color.RED)
                elif self.agent.map_state.map[pos] == 5:
                    self.draw_box(x, y, arcade.color.YELLOW)
                elif self.agent.map_state.map[pos] == 6:
                    self.draw_box(x, y, arcade.color.ORANGE)
                else:
                    self.draw_box(x, y, arcade.color.AQUA)

    def on_update(self, delta_time):
        r = self.agent.do()
        if r == REWARD_END_FIGHT:
            self.agent.reset()


In [ ]:
window = DofusStatusWindow()
arcade.run()

In [ ]:
agent = Agent()
agent.score = -1
cnt = 0

while 1:

    agent.reset()
    
    while agent.do() != REWARD_END_FIGHT:
        continue

    cnt += 1
    print(f"Score {cnt} : {agent.score}")
    